In [ ]:
from DataManagement.data_manager import DataManager
from DataManagement.coin_data import CoinData
from TransferEntropy.transfer_entropy import TransferEntropy
from constants import ATR_ABNORMALITY_COLUMN
# libs
import pandas as pd
import numpy as np

# plotting
import seaborn as sn
import matplotlib.pyplot as plt
# plt.rcParams["figure.figsize"] = (70,30)

# mutual information
from sklearn.feature_selection import mutual_info_classif

In [ ]:
# coin_symbol_list = ["BTCUSDT", "ETHUSDT", "ADAUSDT", "LTCUSDT", "BNBUSDT", "XRPUSDT"]
coin_symbol_list = ["BTCUSDT", "ETHUSDT", "ADAUSDT", "LTCUSDT"]
feature = "close_pct"
kline_size = "1h"
main_coin_symbol = "BTCUSDT"
transfer_entropy_lookback = 24

In [ ]:
data_manager = DataManager()
transfer_entropy = TransferEntropy(lookback=transfer_entropy_lookback)

In [ ]:
main_1h_df = data_manager.get_historical_data_CoinData(main_coin_symbol, kline_size="1h").df
main_abnromal_timestamps = main_1h_df[main_1h_df[ATR_ABNORMALITY_COLUMN] == 1].index

In [ ]:
coin_data_list = [data_manager.get_historical_data_CoinData(coin_symbol, kline_size="1m") for coin_symbol in coin_symbol_list]
one_hour_coin_data_list = [data_manager.get_historical_data_CoinData(coin_symbol, kline_size="1h") for coin_symbol in coin_symbol_list]

# Join close_pct columns of all coins togther

In [ ]:
def join_dataframes(coin_data_list, shift):
    df = pd.DataFrame(index=coin_data_list[0].df.index)

    for coin_data in coin_data_list:
        df = df.join(coin_data.df.shift(shift)[feature], how="outer", rsuffix=f"_{coin_data.coin_symbol}", on="timestamp")

    df.rename(columns={"close_pct":f"close_pct_{coin_data_list[0].coin_symbol}"}, inplace=True)    
    df.dropna(inplace=True)
    df.drop(columns=["timestamp"])
    df.set_index("timestamp", inplace=True)
    return df

In [ ]:
one_min_df = join_dataframes(coin_data_list=coin_data_list, shift=0)
one_hour_df = join_dataframes(one_hour_coin_data_list, shift=0)
one_hour_df = data_manager.add_noise_to_df(one_hour_df)

# Transfer Entropy Test

In [ ]:
# x = random
# yn = xn-1
btc_series = one_hour_df['close_pct_BTCUSDT']
test_df = pd.DataFrame(btc_series)
test_df.rename(columns={"close_pct_BTCUSDT":"btc"}, inplace=True)
test_df['btc_minus_1'] = btc_series.values - 1;
test_df

# Create correl df

In [ ]:
def mutual_information(a, b):
    return mutual_info_regression(a.reshape(-1,1), b, discrete_features = False)[0]

# Transfer Entropy

In [ ]:
# def get_correl_matrix(arr : np.ndarray, method: str = None, column_name_list: list = None, is_symetric: bool = False):
#     cols = arr.shape[1]
#     result = np.ones(shape=(cols,cols))

#     if is_symetric:
#         if method is None:
#             method = "spearman"
#         return pd.DataFrame(arr).corr(method=method)

#     if type(arr) == pd.core.frame.DataFrame:
#         arr = arr.to_numpy()
   
#     if method == "transfer_entropy":
#         method = transfer_entropy.get_transfer_entropy
    
#     for i in range(cols):
#         for j in range(cols):
#             if i == j:
#                 continue
#             arr1 = arr[:,j]
#             arr2 = arr[:,i]
#             print(i, j)
#             result[i,j] = method(arr[i], arr[j])
    
#     if column_name_list is not None:
#         result = pd.DataFrame(data=result, columns=column_name_list, index=column_name_list)
#     return result

In [ ]:
def get_transfer_entropy_matrix(df : pd.DataFrame, method: str = None, is_symetric = False):
    num_cols = df.shape[1]
    result = np.ones(shape=(num_cols, num_cols))

    if method == "transfer_entropy" or method is None:
        method = transfer_entropy.get_transfer_entropy

    if is_symetric:
        if method is None:
            method = "spearman"
        return df.corr(method=method)


    for i in range(num_cols):
        for j in range(num_cols):
            if i==j:
                continue
            result[i,j] = method(df.iloc[:, i].to_numpy(), df.iloc[:, j].to_numpy())
    
    return pd.DataFrame(data=result, columns=df.columns, index=df.columns)

In [ ]:
from scipy.stats import spearmanr
def yeet(arr1, arr2):
    result = spearmanr(arr1, arr2)[0]
    # print(result)
    return result

get_transfer_entropy_matrix(df=one_hour_df, is_symetric=True)

In [ ]:
get_transfer_entropy_matrix(df=one_hour_df)

In [ ]:
def get_transfer_entropy_correl_matrix_list(df):
    window_df_list = list(df.rolling(window=transfer_entropy_lookback))

    alltime_correlation_matrix_list, abnormal_correlation_matrix_list = [],[]
    i = 0
    skipping_counter = 0
    for temp_df in window_df_list: # every df is the 1min data from that hour.
        timestamp = temp_df.index[0]
        if temp_df.empty:
            skipping_counter += 1
            continue
        try:
            temp_correl_matrix = get_transfer_entropy_matrix(df=temp_df).to_numpy()
            
        except:
            skipping_counter += 1
            continue
        
        if np.any(np.isnan(temp_correl_matrix)):
            skipping_counter += 1
            continue
        
        if timestamp in main_abnromal_timestamps:
            abnormal_correlation_matrix_list.append(temp_correl_matrix)
        
        alltime_correlation_matrix_list.append(temp_correl_matrix)

    # transfrom to np.array
    abnormal_correlation_matrix_list = np.array(abnormal_correlation_matrix_list)
    alltime_correlation_matrix_list = np.array(alltime_correlation_matrix_list)
    
    # print skipping precent
    print(f"skipping precent {(skipping_counter/len(window_df_list))*100} %")
    return alltime_correlation_matrix_list, abnormal_correlation_matrix_list

In [ ]:
def get_correl_matrix_list(df, method = 'spearman'):
    if method == 'transfer_entropy':
        print("transfer_entropy")
        return get_transfer_entropy_correl_matrix_list(df)

    resampled_list = list(df.resample(rule="1h"))
    alltime_correlation_matrix_list, abnormal_correlation_matrix_list = [],[]

    skipping_counter = 0
    for timestamp, temp_df in resampled_list: # every df is the 1min data from that hour.
        
        if temp_df.empty:
            skipping_counter += 1
            continue

        try:

            temp_correl_matrix = temp_df.corr(method=method).to_numpy() # df of correlation between columns. 4x4
            
        except:
            skipping_counter += 1
            continue
        
        if np.any(np.isnan(temp_correl_matrix)):
            skipping_counter += 1
            continue
        
        if timestamp in main_abnromal_timestamps:
            abnormal_correlation_matrix_list.append(temp_correl_matrix)
        
        alltime_correlation_matrix_list.append(temp_correl_matrix)

    
    # transfrom to np.array
    abnormal_correlation_matrix_list = np.array(abnormal_correlation_matrix_list)
    alltime_correlation_matrix_list = np.array(alltime_correlation_matrix_list)
    
    # print skipping precent
    print(f"skipping precent {(skipping_counter/len(resampled_list))*100} %")
    return alltime_correlation_matrix_list, abnormal_correlation_matrix_list

# mutual info

In [ ]:
# alltime_mi_matrix_list, abnormal_mi_matrix_list = get_correl_matrix_list(df, method=mutual_information)
# alltime_correlation_matrix_list, abnormal_correlation_matrix_list = get_correl_matrix_list(df)

# correl

In [ ]:
suptitle_fontsize = 40
font_scale = 3

In [ ]:
def plot_heatmap(correl_df, title, shift = 0):
    mean_corr_df = pd.DataFrame(index=coin_symbol_list, columns=coin_symbol_list, data=correl_df)

    fig_size = max(len(df.columns), 10)
    sn.set(rc={"figure.figsize":tuple([fig_size]*2)})
    title = f"Settings - feature:{feature} | time-frame:{kline_size} | main-coin:{main_coin_symbol} | shift:{shift}\n" + title

    sn.heatmap(mean_corr_df, annot=True).set(title=title);

def plot_heatmap_on_axes(data, title, axes):
    fig_size = 10
    sn.set(rc={"figure.figsize":tuple([fig_size]*2)}, font_scale = font_scale)
    
    size = data.shape[0]
    mean_corr_df = pd.DataFrame(index=coin_symbol_list[:size], columns=coin_symbol_list[:size], data=data)
    sn.heatmap(mean_corr_df, annot=True, ax=axes).set(title=title);

In [ ]:
def plot_heatmap_master(df, correl_method = 'spearman', shift = 0):
    alltime_correl_matrix_list, abnormal_correl_matrix_list = get_correl_matrix_list(df, method=correl_method)

    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(70,20))
    fig_title = f"Settings - feature:{feature} | time-frame:{kline_size} | main-coin:{main_coin_symbol} | shift:{shift}"
    fig.suptitle(fig_title, fontsize = suptitle_fontsize)
    
    alltime_correl_mean = alltime_correl_matrix_list.mean(axis=0)
    abnormal_correl_mean = abnormal_correl_matrix_list.mean(axis=0)
    
    title_data_dict = {
        f"alltime Mean of 1h {correl_method}": alltime_correl_mean, 
        f"abnormal Mean of 1h {correl_method}":abnormal_correl_mean, 
        "abnromal minus alltime": abnormal_correl_mean-alltime_correl_mean}
    for i, (title, data) in enumerate(title_data_dict.items()):
        plot_heatmap_on_axes(data=data, title=title, axes=axes[i])
    fig.tight_layout()
    filename = "correlation_research_plot.jpg"
    fig.savefig(filename)

In [ ]:
# plot_heatmap_master(one_hour_df, correl_method='transfer_entropy')

In [ ]:
plot_heatmap_master(test_df, correl_method='transfer_entropy')

# Pearson Correlation

In [ ]:
plot_heatmap(alltime_correlation_matrix_list.mean(axis=0), "all time Mean of 1min correaltion")

In [ ]:
plot_heatmap(abnormal_correlation_matrix_list.mean(axis=0), "Mean of abnomral 1min correlation")

# Difference

In [ ]:
plot_heatmap(abnormal_correlation_matrix_list.mean(axis=0)-alltime_correlation_matrix_list.mean(axis=0), "Difference between all-time and Abnormal")

In [ ]:
plot_heatmap(abnormal_correlation_matrix_list.std(axis=0)-alltime_correlation_matrix_list.std(axis=0), "diff")

# MI

In [ ]:
plot_heatmap(abnormal_mi_matrix_list.mean(axis=0), "all time")

In [ ]:
plot_heatmap(abnormal_mi_matrix_list.mean(axis=0) - alltime_mi_matrix_list.mean(axis=0), "difference in Mean 1min m1")